# Parse Data

In [ ]:
import polars as pl
from typing import List, Tuple

def parse_tase_schema(df: pl.DataFrame):
    result = df.rename({
        'תאריך': 'date',
        'מחיר קניה': 'price'
    }).select('date', 'price').drop_nulls()

    result = result.with_columns(
        pl.col("date").str.strptime(pl.Date,format=r"%d/%m/%Y %H:%M:%S")  # Use the format that matches the input string
        .alias("date")  # Rename the column to store the date
    )

    return result

def find_top_changes(harel_df, descending):
    # Find date of lessening stocks.
    df = harel_df.with_columns(pl.col("price").shift(1).alias("previous_price"))
    df = df.with_columns((pl.col('previous_price') / pl.col('price')).alias('change'))
    return df.sort('change', descending=descending)
    



In [261]:
HAREL_PATH = r'C:\Users\omrik\Documents\Programming\Money\investment_strat\harel_caspit_shiklit.csv'
harel_df = pl.read_csv(HAREL_PATH)
harel_df = parse_tase_schema(harel_df)


# Run this if you want to see the bad data
# find_top_changes(harel_df, True).head(5)
# At some point 2024-11-04 the stocks value was multiplied by 10. We want to normalize this.
harel_df = harel_df.with_columns(
    pl.when(pl.col("price") <= 108)
    .then(pl.col("price") * 10)
    .otherwise(pl.col("price"))
    .alias("price")
)

harel_df = harel_df.with_columns(
    pl.col("price")/100
)

# This shows the data is normalized, and there are no sudden drops/jumps. 
# This is expected since this is a keren caspit.
print(find_top_changes(harel_df, True).head(5))
print(find_top_changes(harel_df, False).head(5))


shape: (5, 4)
┌────────────┬────────┬────────────────┬──────────┐
│ date       ┆ price  ┆ previous_price ┆ change   │
│ ---        ┆ ---    ┆ ---            ┆ ---      │
│ date       ┆ f64    ┆ f64            ┆ f64      │
╞════════════╪════════╪════════════════╪══════════╡
│ 2024-11-20 ┆ 11.027 ┆ null           ┆ null     │
│ 2020-03-19 ┆ 10.021 ┆ 10.034         ┆ 1.001297 │
│ 2022-07-03 ┆ 10.082 ┆ 10.091         ┆ 1.000893 │
│ 2024-06-20 ┆ 10.826 ┆ 10.8356        ┆ 1.000887 │
│ 2022-05-19 ┆ 10.066 ┆ 10.072         ┆ 1.000596 │
└────────────┴────────┴────────────────┴──────────┘
shape: (5, 4)
┌────────────┬────────┬────────────────┬──────────┐
│ date       ┆ price  ┆ previous_price ┆ change   │
│ ---        ┆ ---    ┆ ---            ┆ ---      │
│ date       ┆ f64    ┆ f64            ┆ f64      │
╞════════════╪════════╪════════════════╪══════════╡
│ 2024-11-20 ┆ 11.027 ┆ null           ┆ null     │
│ 2020-03-18 ┆ 10.037 ┆ 10.021         ┆ 0.998406 │
│ 2020-03-15 ┆ 10.074 ┆ 10.058      

In [262]:
YALIN_PATH = r'C:\Users\omrik\Documents\Programming\Money\investment_strat\yalin_lapidot_yeter.csv'
yalin_df = pl.read_csv(YALIN_PATH)
yalin_df = parse_tase_schema(yalin_df)
yalin_df
print(find_top_changes(yalin_df, True).head(5))
print(find_top_changes(yalin_df, False).head(5))


shape: (5, 4)
┌────────────┬────────┬────────────────┬──────────┐
│ date       ┆ price  ┆ previous_price ┆ change   │
│ ---        ┆ ---    ┆ ---            ┆ ---      │
│ date       ┆ f64    ┆ f64            ┆ f64      │
╞════════════╪════════╪════════════════╪══════════╡
│ 2024-11-20 ┆ 642.14 ┆ null           ┆ null     │
│ 2020-03-23 ┆ 276.01 ┆ 292.88         ┆ 1.061121 │
│ 2020-03-25 ┆ 302.74 ┆ 315.84         ┆ 1.043271 │
│ 2020-04-06 ┆ 339.43 ┆ 353.52         ┆ 1.041511 │
│ 2020-03-18 ┆ 273.75 ┆ 284.03         ┆ 1.037553 │
└────────────┴────────┴────────────────┴──────────┘
shape: (5, 4)
┌────────────┬────────┬────────────────┬──────────┐
│ date       ┆ price  ┆ previous_price ┆ change   │
│ ---        ┆ ---    ┆ ---            ┆ ---      │
│ date       ┆ f64    ┆ f64            ┆ f64      │
╞════════════╪════════╪════════════════╪══════════╡
│ 2024-11-20 ┆ 642.14 ┆ null           ┆ null     │
│ 2020-03-08 ┆ 387.88 ┆ 356.61         ┆ 0.919382 │
│ 2020-03-11 ┆ 347.55 ┆ 321.23      

In [277]:
portfolio_df = harel_df.join(yalin_df, on='date', suffix='_yalin').rename({'price':'price_harel'})
portfolio_df = portfolio_df.sort("date")
portfolio_df


shape: (1_230, 3)
┌────────────┬─────────────┬─────────────┐
│ date       ┆ price_harel ┆ price_yalin │
│ ---        ┆ ---         ┆ ---         │
│ date       ┆ f64         ┆ f64         │
╞════════════╪═════════════╪═════════════╡
│ 2019-11-21 ┆ 10.085      ┆ 422.63      │
│ 2019-11-24 ┆ 10.086      ┆ 424.92      │
│ 2019-11-25 ┆ 10.086      ┆ 425.73      │
│ 2019-11-26 ┆ 10.086      ┆ 426.85      │
│ 2019-11-27 ┆ 10.085      ┆ 427.31      │
│ 2019-11-28 ┆ 10.085      ┆ 427.73      │
│ 2019-12-01 ┆ 10.085      ┆ 426.71      │
│ 2019-12-02 ┆ 10.086      ┆ 424.75      │
│ 2019-12-03 ┆ 10.086      ┆ 418.25      │
│ 2019-12-04 ┆ 10.086      ┆ 418.28      │
│ 2019-12-05 ┆ 10.087      ┆ 418.97      │
│ 2019-12-08 ┆ 10.087      ┆ 422.75      │
│ 2019-12-09 ┆ 10.087      ┆ 424.8       │
│ 2019-12-10 ┆ 10.087      ┆ 423.86      │
│ 2019-12-11 ┆ 10.087      ┆ 424.94      │
│ 2019-12-12 ┆ 10.087      ┆ 424.98      │
│ 2019-12-15 ┆ 10.087      ┆ 425.6       │
│ 2019-12-16 ┆ 10.086      ┆ 425.44 

# 

# Run Strategy

In [278]:
# date, price_harel, price_yalin
# earliest date = 2019-11-21
# oldest date = 2024-11-20
strategy_df = portfolio_df.rename({'price_harel': 'price_solid', 'price_yalin': 'price_risky'})

SOLID_ALLOCATION = 20 / 100
RISKY_ALLOCATION = 80 / 100
START_VALUE = 125_000

In [288]:
portfolio_values = []
solid_values = []
risky_values = []
money_in_bank_values = []
money_in_bank = START_VALUE
num_solid_units = 0
num_risky_units = 0

for i in range(len(strategy_df)):
    # Calculate current portfolio value
    portfolio_value = (
        money_in_bank
        + num_solid_units * strategy_df["price_solid"][i]
        + num_risky_units * strategy_df["price_risky"][i]
    )

    # Determine target allocation
    target_solid_value = portfolio_value * SOLID_ALLOCATION
    target_risky_value = portfolio_value * RISKY_ALLOCATION

    # Calculate units to hold for each investment
    num_solid_units = target_solid_value // strategy_df["price_solid"][i]
    num_risky_units = target_risky_value // strategy_df["price_risky"][i]

    # Calculate invested value
    invested_solid = num_solid_units * strategy_df["price_solid"][i]
    invested_risky = num_risky_units * strategy_df["price_risky"][i]
    invested_value = invested_solid + invested_risky

    # Update cash in bank
    money_in_bank = portfolio_value - invested_value

    # Record portfolio details
    portfolio_values.append(portfolio_value)
    solid_values.append(invested_solid)
    risky_values.append(invested_risky)
    money_in_bank_values.append(money_in_bank)

    # # Logging for debugging
    # date = strategy_df["date"][i]
    # print(
    #     f"On {date} Solid: {invested_solid:.2f}, Risky: {invested_risky:.2f}, "
    #     f"Portfolio: {portfolio_value:.2f}, Left in bank: {money_in_bank:.2f}"
    # )

# Add portfolio data to the DataFrame
result_df = strategy_df.with_columns([
    pl.Series("portfolio_value", portfolio_values),
    pl.Series("solid_value", solid_values),
    pl.Series("risky_value", risky_values),
    pl.Series("money_in_bank", money_in_bank_values),
])

print(result_df)


shape: (1_230, 7)
┌────────────┬─────────────┬─────────────┬──────────────┬─────────────┬─────────────┬──────────────┐
│ date       ┆ price_solid ┆ price_risky ┆ portfolio_va ┆ solid_value ┆ risky_value ┆ money_in_ban │
│ ---        ┆ ---         ┆ ---         ┆ lue          ┆ ---         ┆ ---         ┆ k            │
│ date       ┆ f64         ┆ f64         ┆ ---          ┆ f64         ┆ f64         ┆ ---          │
│            ┆             ┆             ┆ f64          ┆             ┆             ┆ f64          │
╞════════════╪═════════════╪═════════════╪══════════════╪═════════════╪═════════════╪══════════════╡
│ 2019-11-21 ┆ 10.085      ┆ 422.63      ┆ 125000.0     ┆ 24990.63    ┆ 99740.68    ┆ 268.69       │
│ 2019-11-24 ┆ 10.086      ┆ 424.92      ┆ 125542.918   ┆ 25104.054   ┆ 100281.12   ┆ 157.744      │
│ 2019-11-25 ┆ 10.086      ┆ 425.73      ┆ 125734.078   ┆ 25144.398   ┆ 100472.28   ┆ 117.4        │
│ 2019-11-26 ┆ 10.086      ┆ 426.85      ┆ 125998.398   ┆ 25194.828   ┆ 1

In [187]:
i=8
current_value = (num_solid_units * strategy_df["price_solid"][i]) + (num_risky_units * strategy_df["price_risky"][i])
current_value

123966.1114208803

In [144]:
strategy_df['price_solid'][405]

1006.5999999999999

0.2